In [15]:
from line_solver import *
import numpy as np
GlobalConstants.setVerbose(VerboseLevel.STD)

In [16]:
# Fork-Join queueing network
model = Network('model')

In [17]:
# Create network 
source = Source(model, 'Source')

queue1 = Queue(model, 'Queue1', SchedStrategy.PS)
queue2 = Queue(model, 'Queue2', SchedStrategy.PS)

fork = Fork(model, 'Fork')
fork.setTasksPerLink(2)
join = Join(model, 'Join', fork)

sink = Sink(model, 'Sink')

In [18]:
# Create job classes
jobclass1 = OpenClass(model, 'class1')
jobclass2 = OpenClass(model, 'class2')

In [19]:
#Set arrival and service processes
# Class 1
source.setArrival(jobclass1, Exp(0.25))
queue1.setService(jobclass1, Exp(1.0))
queue2.setService(jobclass1, Exp(0.75))
# Class 2
source.setArrival(jobclass2, Exp(0.25))
queue1.setService(jobclass2, Immediate())
queue2.setService(jobclass2, Exp(2.0))

In [20]:
# Set routing matrix
P = model.initRoutingMatrix() # Class 1 routing
P.set(jobclass1, jobclass1, source, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)
P.set(jobclass1, jobclass1, join, sink, 1.0) # Class 2 routing
P.set(jobclass2, jobclass2, source, fork, 1.0)
P.set(jobclass2, jobclass2, fork, queue1, 1.0)
P.set(jobclass2, jobclass2, fork, queue2, 1.0)
P.set(jobclass2, jobclass2, queue1, join, 1.0)
P.set(jobclass2, jobclass2, queue2, join, 1.0)
P.set(jobclass2, jobclass2, join, sink, 1.0)
model.link(P)

In [21]:
# Solve with multiple methods
solvers = [ SolverJMT(model, 'seed', 23000), SolverMVA(model)]
for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.getName()}')

avgTable = solver.getAvgTable()

SOLVER 1: SolverJMT
SOLVER 2: SolverMVA
MVA [method: default(egflin), lang: java, env: 21.0.7] completed in 0.173000s.
  Station JobClass     QLen    Util       RespT   ResidT  ArvR  Tput
0  Source   class1   0.0000  0.0000  0.0000e+00   0.0000   0.0  0.25
1  Source   class2   0.0000  0.0000  0.0000e+00   0.0000   0.0  0.25
2  Queue1   class1   1.0000  0.5000  2.0000e+00   1.0000   0.5  0.50
3  Queue1   class2   0.0000  0.0000  2.0000e-08   0.0000   0.5  0.50
4  Queue2   class1   7.9989  0.6667  1.5998e+01   7.9990   0.5  0.50
5  Queue2   class2   2.9996  0.2500  5.9992e+00   2.9996   0.5  0.50
6    Join   class1  23.4382  0.0000  2.3438e+01  23.4385   1.0  0.25
7    Join   class2   8.9976  0.0000  8.9977e+00   8.9977   1.0  0.25
